# Explorative Datenanalyse - CDA2/2Da

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def import_swiss_votes_data(filepath):
    """
    Import Swiss votes data from a CSV file.
    
    Parameters:
    filepath (str): Path to the CSV file
    
    Returns:
    pandas.DataFrame: dataframe with a subselect of csv data
    """
    
    # Import the CSV data
    # The data doesn't have headers and uses semicolons as separators
    df = pd.read_csv(filepath, sep=';')
    
    # Based on the data structure, we can create appropriate column names
    # This is a simplified set of column names based on the visible patterns
    columns_to_use = [
        'datum', 'titel_kurz_d', 'titel_off_d', 'stichwort', 'swissvoteslink', 'rechtsform', 'd1e1', 'd1e2', 'd1e3', 
        'dep', 'br-pos', 'bv-pos', 'nr-pos', 'sr-pos', 'annahme', 'berecht', 'stimmen', 'bet', 'leer', 'ungultig', 'gultig', 'volkja',
        'volknein', 'bfsdash-de', 'bfsmap-de', 'nach_cockpit_d',

        # Kantonale Parameter
        'zh-ja', 'be-ja', 'zg-ja', 'so-ja', 'ag-ja', 'ti-ja', 'vs-ja', 'bs-ja', 'bl-ja',
        'ju-ja', 'ne-ja', 'ow-ja', 'nw-ja', 'ai-ja', 'sg-ja', 'tg-ja', 'ar-ja', 'gl-ja',
        'gr-ja', 'sh-ja', 'vd-ja', 'ge-ja', 'fr-ja', 'ur-ja', 'lu-ja', 'sz-ja',

        'zh-nein', 'be-nein', 'zg-nein', 'so-nein', 'ag-nein', 'ti-nein', 'vs-nein', 'bs-nein', 'bl-nein',
        'ju-nein', 'ne-nein', 'ow-nein', 'nw-nein', 'ai-nein', 'sg-nein', 'tg-nein', 'ar-nein', 'gl-nein',
        'gr-nein', 'sh-nein', 'vd-nein', 'ge-nein', 'fr-nein', 'ur-nein', 'lu-nein', 'sz-nein',

        'zh-stimmen', 'be-stimmen', 'zg-stimmen', 'so-stimmen', 'ag-stimmen', 'ti-stimmen', 'vs-stimmen', 'bs-stimmen', 'bl-stimmen',
        'ju-stimmen', 'ne-stimmen', 'ow-stimmen', 'nw-stimmen', 'ai-stimmen', 'sg-stimmen', 'tg-stimmen', 'ar-stimmen', 'gl-stimmen',
        'gr-stimmen', 'sh-stimmen', 'vd-stimmen', 'ge-stimmen', 'fr-stimmen', 'ur-stimmen', 'lu-stimmen', 'sz-stimmen',

        'zh-japroz', 'be-japroz', 'zg-japroz', 'so-japroz', 'ag-japroz', 'ti-japroz', 'vs-japroz', 'bs-japroz', 'bl-japroz',
        'ju-japroz', 'ne-japroz', 'ow-japroz', 'nw-japroz', 'ai-japroz', 'sg-japroz', 'tg-japroz', 'ar-japroz', 'gl-japroz',
        'gr-japroz', 'sh-japroz', 'vd-japroz', 'ge-japroz', 'fr-japroz', 'ur-japroz', 'lu-japroz', 'sz-japroz',
    ]
    # srja, srnein, nrja, nrnein
    # Parteiparolen könnten spannend sein, p-fdp, p-sps, p-svp, p-mitte, etc.

    df_subset = df[columns_to_use]
    
    print(f"Cleaned data: {df.shape[0]} rows, {df.shape[1]} columns")
    return df_subset

swissvotes_data_path = '../data/raw/swissvotes_data.csv'

swissvotes_data_subset = import_swiss_votes_data(swissvotes_data_path)

In [ ]:
swissvotes_data_subset.head()

In [ ]:
def clean_and_prepare_data(df_to_clean):
    
    # 1. Convert date to datetime format
    # Transform dates from DD.MM.YYYY to YYYY-MM-DD format
    df_to_clean['datum'] = pd.to_datetime(df_to_clean['datum'], format='%d.%m.%Y', errors='coerce')
    
    # 2. Replace placeholder values with NaN
    # In the data, '.' is used as unknown and '9999' appears to be not applicable (organisation doesn't exist)
    # We will be replaceing all these values with nan
    placeholder_values = ['.', '9999', '']
    for placeholder in placeholder_values:
        df_to_clean.replace(placeholder, np.nan, inplace=True)
    
    # 3. Convert boolean-like columns (0/1) to boolean
    boolean_cols = [col for col in df_to_clean.columns if 
                    df_to_clean[col].dropna().astype(str).str.match(r'^[01]$').all()]
    
    for col in boolean_cols:
        df_to_clean[col] = df_to_clean[col].map({0: False, '0': False, 1: True, '1': True})
    
    # 4. Strip whitespace from string columns
    string_cols = df_to_clean.select_dtypes(include=['object']).columns
    for col in string_cols:
        df_to_clean[col] = df_to_clean[col].astype(str).str.strip()
        # Convert back to object type if the column had empty strings that were converted to 'nan'
        df_to_clean[col] = df_to_clean[col].replace('nan', np.nan)
    
    # 6. Handle URLs - validate and clean
    url_cols = [col for col in df_to_clean.columns if 'url' in col.lower()]
    for col in url_cols:
        if col in df_to_clean.columns:
            # Basic URL validation - ensure URLs start with http
            mask = df_to_clean[col].notna() & ~df_to_clean[col].astype(str).str.startswith(('http://', 'https://'))
            if mask.any():
                print(f"Warning: Found {mask.sum()} invalid URLs in column {col}")
                
    # 7. Remove completely empty rows
    df_to_clean = df_to_clean.dropna(how='all')
    
    # 8. Handle trailing semicolons that might have created extra empty columns
    # Drop columns that are completely empty
    df_to_clean = df_to_clean.dropna(axis=1, how='all')

    return df_to_clean

In [ ]:
swissvotes_data_subset_cleaned = clean_and_prepare_data(swissvotes_data_subset)

swissvotes_data_subset_cleaned.head()

## Vorbereitung Daten für die Darstellung der Schweiz

In [ ]:
# Based on article: https://chrimaform.medium.com/switzerland-f35a175d1380
# Data Set from: https://github.com/smalltoe/medium-articles/tree/main/article2_Plotly_Choropleth_for_Switzerland/maps

import geopandas as gpd
import pyproj
from shapely.ops import transform

f=r"..\data\maps\swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp"
shapes = gpd.read_file(f)
shapes.geometry=shapes.geometry.simplify(0.5)

lv03 = pyproj.CRS('EPSG:21781')
wgs84 = pyproj.CRS('EPSG:4326')
project = pyproj.Transformer.from_crs(lv03, wgs84, 
                                          always_xy=True).transform

for i, shape in shapes.iterrows():
    if shape.geometry is not None:
        shape.geometry = transform(project, shape.geometry)
        shapes.iloc[i] = shape

shapes.head()

In [ ]:
# Wichtig der Datensatz hat per Kanton mehrer Einträge wegen Exklaven
print(shapes['NAME'])

# Zuweisung Ids per Kanton
Aargau = ['9', '47']
Appenzell_Ausserrhoden = ['22']
Appenzell_Innerrhoden = ['24', '34', '39', '49', '50']
Basel_Landschaft = ['17', '36']
Basel_Stadt = ['28']
Bern = ['1', '38', '41', '44']
Fribourg = ['7', '25', '31', '37', '43', '48']
Geneve = ['20', '35', '45']
Glarus = ['16']
Graubuenden = ['0']
Jura = ['13']
Luzern = ['8']
Neuchatel = ['14']
Nidwalden = ['19']
Obwalden = ['18', '27']
Schaffhausen = ['21', '29', '33']
Schwyz = ['12']
Solothurn = ['15', '30', '32', '42']
St_Gallen = ['5', '46']
Ticino = ['4']
Thurgau = ['11', '40']
Uri = ['10']
Valais = ['2']
Vaud = ['3', '26']
Zuerich = ['6']
Zug = ['23']

## Importieren und Vorbereitung der Daten der Religionszugehörigkeit

In [ ]:
# === 1. CSV-Datei laden (korrekter Pfad & Komma als Trennzeichen) ===
file_path = '../data/raw/rel_gemeinde.csv'
df = pd.read_csv(file_path, sep=",", encoding="utf-8")

#--Wrangling
# === 2. Leere/unnötige Spalten entfernen ===
df = df.loc[:, ~df.columns.str.match(r"^Unnamed")]

# === 3. In Long-Format bringen ===
id_vars = ['Regions_ID', 'Regionsname', 'Kanton']
value_vars = [col for col in df.columns if col.startswith("y_")]

df_long = pd.melt(df,
                  id_vars=id_vars,
                  value_vars=value_vars,
                  var_name="Jahr",
                  value_name="Religion")

# === 4. Jahr bereinigen (z. B. y_1850 → 1850 als int) ===
df_long["Jahr"] = df_long["Jahr"].str.extract(r"y_(\d+)").astype(int)

# === 5. Encoding-Fehler beheben (‚â• → ≥) ===
df_long["Religion"] = df_long["Religion"].str.replace("‚â•", "≥", regex=False)

# === 6. Nur gültige Kantone (z. B. ZH, BE, AG etc.) ===
df_long["Kanton"] = df_long["Kanton"].astype(str).str.strip()
df_long = df_long[df_long["Kanton"].str.match(r"^[A-Z]{2}$")]

# === 7. Aggregation: häufigste Religion pro Kanton & Jahr ===
df_kanton_agg = (
    df_long
    .groupby(["Kanton", "Jahr"])["Religion"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    .reset_index()
)

print(df_kanton_agg['Jahr'].unique())

In [ ]:
def getReligiousDistributionByYear(year):
    religious_distribution_of_year = df_kanton_agg[df_kanton_agg['Jahr'] == year]

    if (religious_distribution_of_year.empty):
        print (f"No Entry found for the corresponding date {year}")
        return

    ids = [*Aargau,*Appenzell_Ausserrhoden,*Appenzell_Innerrhoden,*Basel_Landschaft,*Basel_Stadt,*Bern,*Fribourg,*Geneve,*Glarus,*Graubuenden,*Jura,*Luzern,*Neuchatel,*Nidwalden,*Obwalden,*Schaffhausen,*Schwyz,*Solothurn,*St_Gallen,*Ticino,*Thurgau,*Uri,*Valais,*Vaud,*Zuerich,*Zug]
    
    distributions = [*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'AG']['Religion'], len(Aargau)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'AR']['Religion'], len(Appenzell_Ausserrhoden)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'AI']['Religion'], len(Appenzell_Innerrhoden)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'BL']['Religion'], len(Basel_Landschaft)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'BS']['Religion'], len(Basel_Stadt)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'BE']['Religion'], len(Bern)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'FR']['Religion'], len(Fribourg)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'GE']['Religion'], len(Geneve)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'GL']['Religion'], len(Glarus)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'GR']['Religion'], len(Graubuenden)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'JU']['Religion'], len(Jura)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'LU']['Religion'], len(Luzern)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'NE']['Religion'], len(Neuchatel)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'NW']['Religion'], len(Nidwalden)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'OW']['Religion'], len(Obwalden)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'SH']['Religion'], len(Schaffhausen)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'SZ']['Religion'], len(Schwyz)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'SO']['Religion'], len(Solothurn)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'SG']['Religion'], len(St_Gallen)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'TI']['Religion'], len(Ticino)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'TG']['Religion'], len(Thurgau)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'UR']['Religion'], len(Uri)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'VS']['Religion'], len(Valais)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'VD']['Religion'], len(Vaud)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'ZH']['Religion'], len(Zuerich)),*np.repeat(religious_distribution_of_year[religious_distribution_of_year['Kanton'] == 'ZG']['Religion'], len(Zug))]

    cantons = [*np.repeat('Aargau', len(Aargau)),*np.repeat('Appenzell_Ausserrhoden', len(Appenzell_Ausserrhoden)),*np.repeat('Appenzell_Innerrhoden', len(Appenzell_Innerrhoden)),*np.repeat('Basel_Landschaft', len(Basel_Landschaft)),*np.repeat('Basel_Stadt', len(Basel_Stadt)),*np.repeat('Bern', len(Bern)),*np.repeat('Fribourg', len(Fribourg)),*np.repeat('Geneve', len(Geneve)),*np.repeat('Glarus', len(Glarus)),*np.repeat('Graubuenden', len(Graubuenden)),*np.repeat('Jura', len(Jura)),*np.repeat('Luzern', len(Luzern)),*np.repeat('Neuchâtel', len(Neuchatel)),*np.repeat('Nidwalden', len(Nidwalden)),*np.repeat('Obwalden', len(Obwalden)),*np.repeat('Schaffhausen', len(Schaffhausen)),*np.repeat('Schwyz', len(Schwyz)),*np.repeat('Solothurn', len(Solothurn)),*np.repeat('St_Gallen', len(St_Gallen)),*np.repeat('Ticino', len(Ticino)),*np.repeat('Thurgau', len(Thurgau)),*np.repeat('Uri', len(Uri)),*np.repeat('Valais', len(Valais)),*np.repeat('Vaud', len(Vaud)),*np.repeat('Zuerich', len(Zuerich)),*np.repeat('Zug', len(Zug))]
    
    # np.repeat(array, n_times) can be used for the reocurring arrays
    data_plot_religious_distribution_visualized = {
        'ids': ids,
        'religious_distribution': distributions,
        'cantons': cantons
    }

    return data_plot_religious_distribution_visualized

## Auswertung der Daten einer Abstimmung

In [ ]:
def getPreparedDataFrameForVisualization(vote_to_be_analyzed):
    # Preparation of the Dataset
    cols = list(vote_to_be_analyzed.filter(like='-japroz').columns)
    vote_visualized = vote_to_be_analyzed[['datum', 'titel_off_d']+cols]

    print(vote_visualized)
    # ,*Jura missing for the first vote that is analyzed
    
    ids = [*Aargau,*Appenzell_Ausserrhoden,*Appenzell_Innerrhoden,*Basel_Landschaft,*Basel_Stadt,*Bern,*Fribourg,*Geneve,*Glarus,*Graubuenden,*Jura,*Luzern,*Neuchatel,*Nidwalden,*Obwalden,*Schaffhausen,*Schwyz,*Solothurn,*St_Gallen,*Ticino,*Thurgau,*Uri,*Valais,*Vaud,*Zuerich,*Zug]
    
    yes_percentages = [*np.repeat(float(vote_visualized['ag-japroz']), len(Aargau)),*np.repeat(float(vote_visualized['ar-japroz']), len(Appenzell_Ausserrhoden)),*np.repeat(float(vote_visualized['ai-japroz']), len(Appenzell_Innerrhoden)),*np.repeat(float(vote_visualized['bl-japroz']), len(Basel_Landschaft)),*np.repeat(float(vote_visualized['bs-japroz']), len(Basel_Stadt)),*np.repeat(float(vote_visualized['be-japroz']), len(Bern)),*np.repeat(float(vote_visualized['fr-japroz']), len(Fribourg)),*np.repeat(float(vote_visualized['ge-japroz']), len(Geneve)),*np.repeat(float(vote_visualized['gl-japroz']), len(Glarus)),*np.repeat(float(vote_visualized['gr-japroz']), len(Graubuenden)),*np.repeat(float(vote_visualized['ju-japroz']), len(Jura)),*np.repeat(float(vote_visualized['lu-japroz']), len(Luzern)),*np.repeat(float(vote_visualized['ne-japroz']), len(Neuchatel)),*np.repeat(float(vote_visualized['nw-japroz']), len(Nidwalden)),*np.repeat(float(vote_visualized['ow-japroz']), len(Obwalden)),*np.repeat(float(vote_visualized['sh-japroz']), len(Schaffhausen)),*np.repeat(float(vote_visualized['sz-japroz']), len(Schwyz)),*np.repeat(float(vote_visualized['so-japroz']), len(Solothurn)),*np.repeat(float(vote_visualized['sg-japroz']), len(St_Gallen)),*np.repeat(float(vote_visualized['ti-japroz']), len(Ticino)),*np.repeat(float(vote_visualized['tg-japroz']), len(Thurgau)),*np.repeat(float(vote_visualized['ur-japroz']), len(Uri)),*np.repeat(float(vote_visualized['vs-japroz']), len(Valais)),*np.repeat(float(vote_visualized['vd-japroz']), len(Vaud)),*np.repeat(float(vote_visualized['zh-japroz']), len(Zuerich)),*np.repeat(float(vote_visualized['zg-japroz']), len(Zug))]
    
    cantons = [*np.repeat('Aargau', len(Aargau)),*np.repeat('Appenzell_Ausserrhoden', len(Appenzell_Ausserrhoden)),*np.repeat('Appenzell_Innerrhoden', len(Appenzell_Innerrhoden)),*np.repeat('Basel_Landschaft', len(Basel_Landschaft)),*np.repeat('Basel_Stadt', len(Basel_Stadt)),*np.repeat('Bern', len(Bern)),*np.repeat('Fribourg', len(Fribourg)),*np.repeat('Geneve', len(Geneve)),*np.repeat('Glarus', len(Glarus)),*np.repeat('Graubuenden', len(Graubuenden)),*np.repeat('Jura', len(Jura)),*np.repeat('Luzern', len(Luzern)),*np.repeat('Neuchâtel', len(Neuchatel)),*np.repeat('Nidwalden', len(Nidwalden)),*np.repeat('Obwalden', len(Obwalden)),*np.repeat('Schaffhausen', len(Schaffhausen)),*np.repeat('Schwyz', len(Schwyz)),*np.repeat('Solothurn', len(Solothurn)),*np.repeat('St_Gallen', len(St_Gallen)),*np.repeat('Ticino', len(Ticino)),*np.repeat('Thurgau', len(Thurgau)),*np.repeat('Uri', len(Uri)),*np.repeat('Valais', len(Valais)),*np.repeat('Vaud', len(Vaud)),*np.repeat('Zuerich', len(Zuerich)),*np.repeat('Zug', len(Zug))]
    
    # np.repeat(array, n_times) can be used for the reocurring arrays
    data_plot_vote_visualized = {
        'ids': ids,
        'ja_stimmen': yes_percentages,
        'cantons': cantons
    }
    
    return data_plot_vote_visualized


In [ ]:
# We will focus this test case for processing and visulaization of the data on the vote: 
# Innovativer Arbeitnehmenrschutz in den Fabriken
# Bundesgesetz betreffend die Arbeit in den Fabriken (21.20.1877)

vote_to_be_analyzed = swissvotes_data_subset_cleaned[swissvotes_data_subset_cleaned['titel_off_d'] == 'Bundesgesez betreffend die Arbeit in den Fabriken']

vote_to_be_analyzed

In [ ]:
data_plot_vote_visualized = getPreparedDataFrameForVisualization(vote_to_be_analyzed)

import plotly.express as px

colorscale = ["rgb(183, 27, 27)", "rgb(223, 129, 129)", "rgb(229, 239, 228)", "rgb(137, 219, 131)", "rgb(43, 131, 36)"]

fig = px.choropleth_mapbox(data_plot_vote_visualized, geojson=shapes, 
                           locations='ids', 
                           color='ja_stimmen',
                           title="Prozentual Anteil Ja-Stimmen per Kanton - Abstimmung Fabrikgesetz 21-10-1877",
                           color_continuous_scale=colorscale,
                           range_color=([0.0, 100.0]),
                           color_continuous_midpoint=50.0,
                           hover_name='cantons',
                           hover_data='ja_stimmen',
                           mapbox_style="carto-positron",
                           zoom=6.3,
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.75,
                           width=900,
                           height=600
                          )
fig.show()

In [ ]:
# Visualisierung Religion
religious_distribution_1880 = getReligiousDistributionByYear(1880)

fig = px.choropleth_mapbox(religious_distribution_1880, geojson=shapes, 
                           locations='ids', 
                           color='religious_distribution',
                           title="Religiöse Verteilung der Kantone - 1880",
                           color_continuous_scale=colorscale,
                           range_color=([0.0, 100.0]),
                           color_continuous_midpoint=50.0,
                           hover_name='cantons',
                           hover_data='religious_distribution',
                           mapbox_style="carto-positron",
                           zoom=6.3,
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.75,
                           width=1000,
                           height=600
                          )
fig.show()

In [ ]:
vote_public_railways = swissvotes_data_subset_cleaned[swissvotes_data_subset_cleaned['titel_off_d'] == 'Bundesgesetz betreffend die Erwerbung und den Betrieb von Eisenbahnen für Rechnung des Bundes und die Organisation der Verwaltung der schweizerischen Bundesbahnen']

data_plot_vote_public_railways = getPreparedDataFrameForVisualization(vote_public_railways)

colorscale = ["rgb(183, 27, 27)", "rgb(223, 129, 129)", "rgb(229, 239, 228)", "rgb(137, 219, 131)", "rgb(43, 131, 36)"]

fig = px.choropleth_mapbox(data_plot_vote_public_railways, geojson=shapes, 
                           locations='ids', 
                           color='ja_stimmen',
                           title="Prozentual Anteil Ja-Stimmen per Kanton - Abstimmung Gesetz über die schweizer Bundesbahnen 20-02-1898",
                           color_continuous_scale=colorscale,
                           range_color=([0.0, 100.0]),
                           color_continuous_midpoint=50.0,
                           hover_name='cantons',
                           hover_data='ja_stimmen',
                           mapbox_style="carto-positron",
                           zoom=6.3,
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.75,
                           width=1000,
                           height=600
                          )
fig.show()

In [ ]:
# Visualisierung Religion
religious_distribution_1890 = getReligiousDistributionByYear(1890)

fig = px.choropleth_mapbox(religious_distribution_1890, geojson=shapes, 
                           locations='ids', 
                           color='religious_distribution',
                           title="Religiöse Verteilung der Kantone - 1890",
                           color_continuous_scale=colorscale,
                           range_color=([0.0, 100.0]),
                           color_continuous_midpoint=50.0,
                           hover_name='cantons',
                           hover_data='religious_distribution',
                           mapbox_style="carto-positron",
                           zoom=6.3,
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.75,
                           width=1000,
                           height=600
                          )
fig.show()

In [ ]:
vote_peace_politic_without_army = swissvotes_data_subset_cleaned[swissvotes_data_subset_cleaned['titel_off_d'] == 'Volksinitiative «für eine Schweiz ohne Armee und für eine umfassende Friedenspolitik»']

data_plot_vote_peace_politic_without_army = getPreparedDataFrameForVisualization(vote_peace_politic_without_army)

colorscale = ["rgb(183, 27, 27)", "rgb(223, 129, 129)", "rgb(229, 239, 228)", "rgb(137, 219, 131)", "rgb(43, 131, 36)"]

fig = px.choropleth_mapbox(data_plot_vote_peace_politic_without_army, geojson=shapes, 
                           locations='ids', 
                           color='ja_stimmen',
                           title="Prozentual Anteil Ja-Stimmen per Kanton - Abstimmung Initiative für eine Schweiz ohne Armee 26-11-1989",
                           color_continuous_scale=colorscale,
                           range_color=([0.0, 100.0]),
                           color_continuous_midpoint=50.0,
                           hover_name='cantons',
                           hover_data='ja_stimmen',
                           mapbox_style="carto-positron",
                           zoom=6.3,
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.75,
                           width=900,
                           height=600
                          )
fig.show()

In [ ]:
religious_distribution_1990 = getReligiousDistributionByYear(1990)

fig = px.choropleth_mapbox(religious_distribution_1990, geojson=shapes, 
                           locations='ids', 
                           color='religious_distribution',
                           title="Religiöse Verteilung der Kantone - 1990",
                           color_continuous_scale=colorscale,
                           range_color=([0.0, 100.0]),
                           color_continuous_midpoint=50.0,
                           hover_name='cantons',
                           hover_data='religious_distribution',
                           mapbox_style="carto-positron",
                           zoom=6.3,
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.75,
                           width=1000,
                           height=600
                          )
fig.show()

In [ ]:
vote_peace_stop_building_nuclear_power_plants = swissvotes_data_subset_cleaned[swissvotes_data_subset_cleaned['titel_off_d'] == 'Volksinitiative «Stopp dem Atomkraftwerkbau (Moratorium)»']

data_plot_vote_stop_building_nuclear_power_plants = getPreparedDataFrameForVisualization(vote_peace_stop_building_nuclear_power_plants)

fig = px.choropleth_mapbox(data_plot_vote_stop_building_nuclear_power_plants, geojson=shapes, 
                           locations='ids', 
                           color='ja_stimmen',
                           title="Prozentual Anteil Ja-Stimmen per Kanton - Abstimmung Initiative «Stopp dem Atomkraftwerkbau» 23-09-1990",
                           color_continuous_scale=colorscale,
                           range_color=([0.0, 100.0]),
                           color_continuous_midpoint=50.0,
                           hover_name='cantons',
                           hover_data='ja_stimmen',
                           mapbox_style="carto-positron",
                           zoom=6.3,
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.75,
                           width=1000,
                           height=600
                          )
fig.show()

In [ ]:
vote_peace_eurpoean_economic_area = swissvotes_data_subset_cleaned[swissvotes_data_subset_cleaned['titel_off_d'] == 'Bundesbeschluss über den Europäischen Wirtschaftsraum (EWR)']

data_plot_vote_eurpoean_economic_area = getPreparedDataFrameForVisualization(vote_peace_eurpoean_economic_area)

fig = px.choropleth_mapbox(data_plot_vote_eurpoean_economic_area, geojson=shapes, 
                           locations='ids', 
                           color='ja_stimmen',
                           title="Prozentual Anteil Ja-Stimmen per Kanton - Abstimmung Beitritt in Europäischen Wirtschaftsraum 06-12-1992",
                           color_continuous_scale=colorscale,
                           range_color=([0.0, 100.0]),
                           color_continuous_midpoint=50.0,
                           hover_name='cantons',
                           hover_data='ja_stimmen',
                           mapbox_style="carto-positron",
                           zoom=6.3,
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.75,
                           width=1000,
                           height=600
                          )
fig.show()

In [ ]:
vote_peace_against_ripoffs = swissvotes_data_subset_cleaned[swissvotes_data_subset_cleaned['titel_off_d'] == 'Volksinitiative «gegen die Abzockerei»']

data_plot_vote_against_ripoffs = getPreparedDataFrameForVisualization(vote_peace_against_ripoffs)

fig = px.choropleth_mapbox(data_plot_vote_against_ripoffs, geojson=shapes, 
                           locations='ids', 
                           color='ja_stimmen',
                           title="Prozentual Anteil Ja-Stimmen per Kanton - Abstimmung Volksinitiative «gegen die Abzockerei» 03-03-2013",
                           color_continuous_scale=colorscale,
                           range_color=([0.0, 100.0]),
                           color_continuous_midpoint=50.0,
                           hover_name='cantons',
                           hover_data='ja_stimmen',
                           mapbox_style="carto-positron",
                           zoom=6.3,
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.75,
                           width=950,
                           height=600
                          )
fig.show()

In [ ]:
religious_distribution_2014 = getReligiousDistributionByYear(2014)

print(religious_distribution_2014)

fig = px.choropleth_mapbox(religious_distribution_2014, geojson=shapes, 
                           locations='ids', 
                           color='religious_distribution',
                           title="Religiöse Verteilung der Kantone - 2014",
                           range_color=([0.0, 100.0]),
                           color_continuous_midpoint=50.0,
                           hover_name='cantons',
                           hover_data='religious_distribution',
                           mapbox_style="carto-positron",
                           zoom=6.3,
                           center = {"lat": 46.8, "lon": 8.5},
                           opacity=0.75,
                           width=1300,
                           height=600
                          )
fig.show()

## Test Visualizations for later

In [ ]:
#https://www.bfs.admin.ch/bfs/de/home/statistiken/bevoelkerung/erhebungen/esrk.html